Name: Kerem Kazandır

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,precision_score,f1_score,accuracy_score,recall_score,roc_curve,auc
from sklearn.preprocessing import label_binarize
from typing import List

In [2]:
movie = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movie = movie.merge(credit,on='title')

In [4]:
movie = movie.drop(columns=['homepage','production_countries','status','tagline','original_language','original_title','spoken_languages','runtime','popularity','movie_id','production_companies','release_date','id','overview'])

In [5]:
import ast
def convert_genre(obj):
    L = str()
    for i in ast.literal_eval(obj):
        L = L + " genre_" + i["name"].replace(" ","_")
    return L

In [6]:
def convert_key(obj):
    L = str()
    for i in ast.literal_eval(obj):
        L = L + " keyword_" + i["name"].replace(" ","_")
    return L

In [7]:
def convert_crew(obj):
    L = str()
    for i in ast.literal_eval(obj):
        job = i["job"]
        if job == "Executive Producer" or job == "Director" or "Writer":
            L = L + " " + job.replace(" ","_") + i["name"].replace(" ","_")
    return L

In [8]:
def convert_cast(obj):
    L = str()
    length = len(ast.literal_eval(obj))/5
    j = 0
    for i in ast.literal_eval(obj):
        L = L + " cast_" + i["name"].replace(" ","_")
        j +=1
        if j>=length:
            break
    return L

In [9]:
movie = movie[movie['vote_count'] >= 100]

In [10]:
movie['genres'] = movie['genres'].apply(convert_genre)
movie['keywords'] = movie['keywords'].apply(convert_key)
movie['cast'] = movie['cast'].apply(convert_cast)
movie['crew'] = movie['crew'].apply(convert_crew)

In [11]:
movie.sort_values('vote_average',ascending = False).head(5)

,budget,genres,keywords,revenue,title,vote_average,vote_count,cast,crew
1887,25000000,genre_Drama genre_Crime,keyword_prison keyword_corruption keyword_pol...,28341469,The Shawshank Redemption,8.5,8205,cast_Tim_Robbins cast_Morgan_Freeman cast_Bob...,Original_Music_ComposerThomas_Newman Director...
3342,6000000,genre_Drama genre_Crime,keyword_italy keyword_love_at_first_sight key...,245066411,The Godfather,8.4,5893,cast_Marlon_Brando cast_Al_Pacino cast_James_...,DirectorFrancis_Ford_Coppola ScreenplayFranci...
662,63000000,genre_Drama,keyword_support_group keyword_dual_identity k...,100853753,Fight Club,8.3,9413,cast_Edward_Norton cast_Brad_Pitt cast_Meat_L...,Executive_ProducerArnon_Milchan Costume_Desig...
3237,8000000,genre_Thriller genre_Crime,keyword_transporter keyword_brothel keyword_d...,213928762,Pulp Fiction,8.3,8428,cast_John_Travolta cast_Samuel_L._Jackson cas...,DirectorQuentin_Tarantino ScreenplayQuentin_T...
2300,15000000,genre_Fantasy genre_Adventure genre_Animation...,keyword_witch keyword_parents_kids_relationsh...,274925095,Spirited Away,8.3,3840,cast_Rumi_Hiiragi cast_Miyu_Irino cast_Mari_N...,ProducerJohn_Lasseter DirectorHayao_Miyazaki ...


In [12]:
df_hot1 = movie['genres'].str.get_dummies(sep=' ')
df_hot2 = movie['keywords'].str.get_dummies(sep=' ')
#df_hot3 = movie['cast'].str.get_dummies(sep=' ')
#df_hot4 = movie['crew'].str.get_dummies(sep=' ')

In [13]:
only_vote = movie.drop(columns =['genres','keywords','cast','crew',])

In [14]:
df_hot1_value = df_hot1.copy()
df_hot2_value = df_hot2.copy()
#df_hot3_value = df_hot3.copy()
#df_hot4_value = df_hot4.copy()


In [15]:
for column in df_hot1.columns:
    df_hot1_value[column] = df_hot1[column].value_counts()
for column in df_hot2.columns:
    df_hot2_value[column] = df_hot2[column].value_counts()
#for column in df_hot3.columns:
 #   df_hot3_value[column] = df_hot3[column].value_counts()
#for column in df_hot4.columns:
 #   df_hot4_value[column] = df_hot4[column].value_counts()

In [16]:
sorted_row = df_hot1_value.iloc[1]
columns = sorted_row[sorted_row>75].index
df_hot1 = df_hot1[columns]

In [17]:
sorted_row = df_hot2_value.iloc[1]
columns = sorted_row[sorted_row>50].index
df_hot2 = df_hot2[columns]

In [18]:
new_df = pd.concat([df_hot1,df_hot2,only_vote],axis=1)

In [19]:
new_df.sort_values('vote_average',ascending = False).head(5)

,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Drama,genre_Family,genre_Fantasy,genre_History,genre_Horror,...,keyword_superhero,keyword_suspense,keyword_teenager,keyword_violence,keyword_woman_director,budget,revenue,title,vote_average,vote_count
1887,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,25000000,28341469,The Shawshank Redemption,8.5,8205
3342,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,6000000,245066411,The Godfather,8.4,5893
662,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,63000000,100853753,Fight Club,8.3,9413
3237,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,8000000,213928762,Pulp Fiction,8.3,8428
2300,0,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,15000000,274925095,Spirited Away,8.3,3840
